In [3]:

import requests
import datetime
import time
from bs4 import BeautifulSoup
import pickle
import pandas as pd

In [8]:
url = 'https://treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate'
treasuriesPicklePath = '/Users/jetti/data/bonds/treasuries.pkl'

In [4]:
def processBondDay(thisDate, treasuryDay):
    thisDay = []
    for thisRow in treasuryDay:
        thisRow = thisRow.find_all('td')
        thisRow = [i.text for i in thisRow]
        if float(thisRow[-1]) > 0.0: # if end-of-day price is non-zero
            thisDay.append([thisDate] + thisRow)
    return thisDay

def dataframeFromPickleData(data):
    df = pd.DataFrame(data)
    df.columns = ['date','cusip','description','coupon','maturity','call','buy','sell','end-of-day-price']
    df = df.drop(['description','call','buy','sell'], axis=1)
    df['end-of-day-price'] = df['end-of-day-price'].astype('float')
    df['coupon'] = [float(i[0:-1]) for i in df['coupon']]
    df['maturity'] = [datetime.datetime.strptime(i,'%m/%d/%Y') for i in df['maturity']]
    return df

def queryTreasuryDirect(thisDate):
    
    day = thisDate.day
    month = thisDate.month
    year = thisDate.year
    data = {'submit': 'Show Prices', 
            'priceDate.month':str(month), 
            'priceDate.day':str(day), 
            'priceDate.year':str(year)}

    r = requests.post(url, data=data)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.findAll('tr')

    processedDay = processBondDay(thisDate, rows[1:])
    return processedDay

def refreshTreasuryData(startDateString, endDateString, pathToPickle):
    
    with open(pathToPickle, 'rb') as f:
          currentData = pickle.load(f)
        
    df = dataframeFromPickleData(currentData)
    currentDates = set(df['date'])
    startDate = datetime.datetime.strptime(startDateString,'%m/%d/%Y')
    endDate = datetime.datetime.strptime(endDateString,'%m/%d/%Y')

    days = (endDate - startDate).days
    daysInInterval = set([startDate + datetime.timedelta(days=i) for i in range(0,days)])
    daysToGet = [i for i in list(daysInInterval - currentDates) if i.weekday() < 5]

    print('days to get: ' + str(len(daysToGet)))
    for i,thisDate in enumerate(daysToGet):
        if i % 100 == 0:
            print(i)
        thisData = queryTreasuryDirect(thisDate)
        if len(thisData) > 1:
            currentData.extend(thisData)
            
    return currentData

In [9]:
dataList = refreshTreasuryData('9/17/2012', '9/17/2023', treasuriesPicklePath)

days to get: 620
0
100
200
300
400
500
600


In [10]:
with open(treasuriesPicklePath, 'wb') as f:
  pickle.dump(dataList, f)

# Scratch

In [ ]:
'''
url = 'https://treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate'
daysBack = 366
dataList = []
sleepTime = 0.0

for thisDay in range(0, daysBack):
    if thisDay % 100 == 0:
        print(thisDay)
    thisDate = datetime.datetime(2023, 9, 16) + datetime.timedelta(-thisDay)
    if thisDate.weekday() < 5:
        day = thisDate.day
        month = thisDate.month
        year = thisDate.year
        data = {'submit': 'Show Prices', 
                'priceDate.month':str(month), 
                'priceDate.day':str(day), 
                'priceDate.year':str(year)}
    
        r = requests.post(url, data=data)
        soup = BeautifulSoup(r.text, 'html.parser')
        rows = soup.findAll('tr')

        processedDay = processBondDay(thisDate, rows[1:])
        dataList.extend(processedDay)
        time.sleep(sleepTime)

print(len(dataList))
'''

In [3]:

import requests
import datetime
import time
from bs4 import BeautifulSoup
import pickle
import pandas as pd

In [8]:
url = 'https://treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate'
treasuriesPicklePath = '/Users/jetti/data/bonds/treasuries.pkl'

In [4]:
def processBondDay(thisDate, treasuryDay):
    thisDay = []
    for thisRow in treasuryDay:
        thisRow = thisRow.find_all('td')
        thisRow = [i.text for i in thisRow]
        if float(thisRow[-1]) > 0.0: # if end-of-day price is non-zero
            thisDay.append([thisDate] + thisRow)
    return thisDay

def dataframeFromPickleData(data):
    df = pd.DataFrame(data)
    df.columns = ['date','cusip','description','coupon','maturity','call','buy','sell','end-of-day-price']
    df = df.drop(['description','call','buy','sell'], axis=1)
    df['end-of-day-price'] = df['end-of-day-price'].astype('float')
    df['coupon'] = [float(i[0:-1]) for i in df['coupon']]
    df['maturity'] = [datetime.datetime.strptime(i,'%m/%d/%Y') for i in df['maturity']]
    return df

def queryTreasuryDirect(thisDate):
    
    day = thisDate.day
    month = thisDate.month
    year = thisDate.year
    data = {'submit': 'Show Prices', 
            'priceDate.month':str(month), 
            'priceDate.day':str(day), 
            'priceDate.year':str(year)}

    r = requests.post(url, data=data)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.findAll('tr')

    processedDay = processBondDay(thisDate, rows[1:])
    return processedDay

def refreshTreasuryData(startDateString, endDateString, pathToPickle):
    
    with open(pathToPickle, 'rb') as f:
          currentData = pickle.load(f)
        
    df = dataframeFromPickleData(currentData)
    currentDates = set(df['date'])
    startDate = datetime.datetime.strptime(startDateString,'%m/%d/%Y')
    endDate = datetime.datetime.strptime(endDateString,'%m/%d/%Y')

    days = (endDate - startDate).days
    daysInInterval = set([startDate + datetime.timedelta(days=i) for i in range(0,days)])
    daysToGet = [i for i in list(daysInInterval - currentDates) if i.weekday() < 5]

    print('days to get: ' + str(len(daysToGet)))
    for i,thisDate in enumerate(daysToGet):
        if i % 100 == 0:
            print(i)
        thisData = queryTreasuryDirect(thisDate)
        if len(thisData) > 1:
            currentData.extend(thisData)
            
    return currentData

In [9]:
dataList = refreshTreasuryData('9/17/2012', '9/17/2023', treasuriesPicklePath)

days to get: 620
0
100
200
300
400
500
600


In [10]:
with open(treasuriesPicklePath, 'wb') as f:
  pickle.dump(dataList, f)

# Scratch

In [ ]:
'''
url = 'https://treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate'
daysBack = 366
dataList = []
sleepTime = 0.0

for thisDay in range(0, daysBack):
    if thisDay % 100 == 0:
        print(thisDay)
    thisDate = datetime.datetime(2023, 9, 16) + datetime.timedelta(-thisDay)
    if thisDate.weekday() < 5:
        day = thisDate.day
        month = thisDate.month
        year = thisDate.year
        data = {'submit': 'Show Prices', 
                'priceDate.month':str(month), 
                'priceDate.day':str(day), 
                'priceDate.year':str(year)}
    
        r = requests.post(url, data=data)
        soup = BeautifulSoup(r.text, 'html.parser')
        rows = soup.findAll('tr')

        processedDay = processBondDay(thisDate, rows[1:])
        dataList.extend(processedDay)
        time.sleep(sleepTime)

print(len(dataList))
'''